In [1]:
import scipy.io as sio
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# Crowder Single Unit Data

### Data Source
These data were provided courtesy of Dr. Nathan Crowder, Department of Psychology & Neuroscience, Dalhousie University. Data are from 23 mouse primary visual cortex neurons; for simplicity we are going to work with data from only 6 of these neurons.

### Experimental Design
- Visual stimuli were sine wave gratings drifting (oscillating) at 2 Hz (cycles/sec)
- Each trial was 4000 ms in duration. 
- Data are sampled at 1000 Hz, meaning that we have a data point every 1 ms.
- The stimuli were presented under 2 experimental **conditions**: control (CTRL) and adaptation (ADAPT). 
    - In the **CTRL** condition, each trial began with a 2000 ms blank grey screen, then the oscillating sine wave grating at the trial's contrast level (see below) for 1000 ms, then 1000 ms blank grey screen.
    - In the **ADAPT** condition, each trial started with the adaptation stimulus, a 50% contrast sine wave grating for 2000 ms, then the sine wave grating at the trial's contrast level (see below), then 1000 ms blank grey screen. 
- Another variable that was manipulated was **contrast** of the sine wave grating. There were 10 contrast levels: 4, 8, 12, 16, 24, 32, 48, 64, 84, 100%. Contrast refers to the difference between the intensities (luminance) of the maximum (white) and minimum (black). Zero contrast would be a uniform grey stimulus (no different between max and minimum intensity), while 100% contrast would be the brightest white versus the blackest black that the monitor can produce.
- within each condition (CTRL, ADAPT), each contrast level was presented 8 times. 

In the next cell we define these as variables that we'll use later in the script, but here we'll also use them to visualize the experimental design:

### Define experiment parameters
Things we know about the experimental design that are useful to hard-code:

In [2]:
cond_labels = ['CTRL', 'ADAPT'] 
contr_labels = [4, 8, 12, 16, 24, 32, 48, 64, 84, 100]
rep_labels = list(np.arange(1, 8))
num_reps = len(rep_labels)

time_labels = list(np.arange(4000))

stim_on_time = 2000
stim_off_time = stim_on_time + 1000

adapt_on_time = 0
adapt_off_time = adapt_on_time + 2000

### Plot Experimental Design

Some of the code below may come in handy later!

In [0]:
"""
fig = plt.figure(figsize=[8,18])

subplot_counter = 1 # used to track subplots
for contr in contr_labels:
    for cond in cond_labels:
        ax = fig.add_subplot(len(contr_labels), len(cond_labels), subplot_counter)
        for rep in rep_labels:
            plt.axhline(rep - .5, 0, 4000, color='black', linestyle='--', linewidth=.75)

        # Show adaptation grating at 50% contrast
        if cond == 'ADAPT':
            plt.axvspan(0, stim_on_time-1, alpha=0.5, color='grey')
                                                          
        # shading indicates stimulus on period
        plt.axvspan(stim_on_time, stim_off_time, alpha=contr/100, color='grey')
        
        # Pretty formatting
        plt.xlim([0, max(time_labels)+1])
        plt.ylim([0, len(rep_labels)])
        plt.title(cond + ' ' + str(contr) + '% contrast')
        plt.xlabel('Time (ms)')
        plt.ylabel('Repetition',)
        plt.yticks([x + 0.5 for x in range(num_reps)], [str(x + 1) for x in range(num_reps)], size=8) 
        plt.tight_layout()
        subplot_counter += 1
        
plt.show()
"""

### Predictions 
- Some primary visual cortex cells are phase sensitive (so-called **simple cells**), so in response to a drifting sine wave grating they will oscillate between excitation and inhibition at the same temporal frequency of the stimulus (2 Hz in this case). There are a few simple cells among the 23 included.
- Stimulus contrast can be plotted against the mean spike rate (2000–3000 ms in this data) to produce a contrast response function, which is a roughly sigmoid shaped function.
- If you plot control and adapted contrast response functions you will find that adaptation "squishes" the contrast responses downwards (firing is attenuated following adaptation) and rightward (it takes higher contrasts to elicit a desired spike rate).
- You can measure neural latency in the control condition, and latency is expected to decrease with increasing contrast.

### Data Structure
Data are provided in a Matlab file, similar to the one used in the [*Correlating Spike Trains*](https://dalpsychneuro.github.io/NESC_3505_textbook/spike_trains/corr_spike_trains.html) chapter. Although I've provided the code below to properly convert these data into a pandas DataFrame, I'll describe the structure briefly, both for reference and because it can help you in thinking about this complex, multi-dimensional data set. 

The data set is 5 dimensional. This is far less sci-fi than it sounds! It just means that we call each variable a "dimension" along which the data vary in a systematic way. So our dimensions are neuron (23 levels) x condition (2 levels) x contrast level (10) x repetitions (8) x time (4000 levels). This means we can compute the expected number of data points to be:

In [0]:
23 * 2 * 10 * 8 * 4000

Whoa... 14.72 million data points! Good times to come. But as noted we're only going to work with 4 neurons, which will save memory and make things more manageable.

In [0]:
4 * 2 * 10 * 8 * 4000

Yep, only 2.5 m data points. Totally manageable. 

One last thing to note, that follows from the 4000 time points, is that these data are not in the form of spike times, but rather times series in which each time point is represented as a 0 (not spiking) or 1 (spiking).

## Read in Matlab data file

In [3]:
dat = sio.loadmat('crowder_single_unit.mat')

## Figure out the structure of the data

<font color='#0F4C81'>
<h2> 
    Q1
    </h2>
What is the type of `dat`? (answer using code output)
</font>

In [0]:
type(dat)

<font color='#0F4C81'>
<h2> 
    Q2
    </h2>
What is the length of `dat`? (answer using code output)
</font>

In [0]:
len(dat)

In the Matlab file, the data are in 'SaveForAaron_May11_2020' (named after the original name that Dr. Crowder gave to the file when sharing it):

In [0]:
dat.keys()

<font color='#0F4C81'>
<h2> 
    Q3
    </h2>
What is the type of `'SaveForAaron_May11_2020'`? (answer using code output)
</font>

In [0]:
type(dat['SaveForAaron_May11_2020'])

<font color='#0F4C81'>
<h2> 
    Q3b
    </h2>
What is the shape of `'SaveForAaron_May11_2020'`? (answer using code output)
</font>

In [0]:
dat['SaveForAaron_May11_2020'].shape

To see the what's stored for each neuron, we need to index the row (always 0, since there's only one row), and the appropriate column for that neuron. So for the first neuron:

In [0]:
dat['SaveForAaron_May11_2020'][0,0]

<font color='#0F4C81'>
<h2> 
    Q4
    </h2>
Modify the command in the previous code cell, to visualize the data from the last neuron in the set:
</font>

In [0]:
dat['SaveForAaron_May11_2020'][-1,-1]

You can see that each of those entries is entirely contained in parentheses, indicating it's a tuple. So rather than the numpy `.shape()` method we need to use the `len()` function:

In [0]:
len(dat['SaveForAaron_May11_2020'][0,0])

The first entry in the tuple is the label of the neuron. Somewhat awkwardly, the label is actually buried inside a numpy array:

In [0]:
dat['SaveForAaron_May11_2020'][0,0][0]

So we have another level of indexing to grab the label:

In [0]:
dat['SaveForAaron_May11_2020'][0,0][0][0]

The second entry in each neuron's tuple is the data:

In [0]:
dat['SaveForAaron_May11_2020'][0,0][1].shape

One confusing thing is that although we have a 5 dimensional experimental design (23 x 2 x 10 x 8 x 4000) our data only has 4 dimensions — the cells, and within each cell, a 3-dimensional array. Furthermore there is no dimension of either length 2 (conditions) or 10 (contrast levels). There *is* however a dimensions with 20 levels, so it looks like those two variables are combined in the data. Indeed, Dr. Crowder documented that the order of those 20 levels is all 10 contrast levels for the CTRL condition first, then all 10 contrast levels for the ADAPT condition. 

Also, in contrast the hierarchical ordering of the variables I provided above (time inside repetitions inside contrast inside condition inside neuron), the data are ordered a bit differently: for each neuron, there are 4000 rows (time) x 8 columns (repetitions) x 20 "pages" (conditions). We call the third dimension of the matrix "pages"; if you think of the data as a 3D cube, visualize this dimension as depth.

## Convert to pandas DataFrame
### Create lists of condition labels

The next cell sets up labels for the rows of the DataFrame we're going to create. It's always important, in multi-dimensional data like this, to know what variables change "fastest" and which change "slowest". "Fastest means, from one row to the next, the value of that variable changes. In our case, we are going to organize the DataFrame so that all of the data for one neuron come before any data from the next neuron. So `neuron` changes slowest. `time` will change fastest, because we want all the time points of one trial (`repetition`), in sequence, before any time points of the next trial. `repetition` changes next-fastest, followed by `contrast` level, followed by `condition`.

Below this we use numpy's `.reshape()` method with the `newshape` argument set to `-1` (to reshape the 3D matrix into a 1D vector) and  `order='F'`. The latter tells the method to read / write the elements with the first index (dimension of the input data) changing fastest, and the last index changing slowest (if you're curious, the `F` is for Fortran, a language which uses this ordering). In our case, the first index is `time`. We know this because the shape of the input data is (4000, 8, 20), so time is the first dimension of the data. and it makes sense to list all the time points for one trial before moving on to the next. We do need to be aware of this so that we can set up the labels for each data point correctly. I've done this for you though, because getting it right is tricky, and if you get it wrong, all your results will be wrong (speaking as someone who learned the hard way...). 

In [4]:
# Compute the total number of data points per neuron; we need to know how
#   many rows our DataFrame needs to have.
len_data = [(x * y * z) for x, y, z in [dat['SaveForAaron_May11_2020'][0][0][1].shape]][0]

# Set up vectors to label the columns in the pandas DataFrame
num_tp = dat['SaveForAaron_May11_2020'][0,0][1].shape[0]
time_labels = list(np.arange(num_tp))
times = time_labels * (len_data//len(time_labels))

num_condcontr = dat['SaveForAaron_May11_2020'][0,0][1].shape[2]
num_cond = len(cond_labels)
num_contr = len(contr_labels)

num_reps = dat['SaveForAaron_May11_2020'][0,0][1].shape[1]
rep_labels = list(np.arange(1, num_reps+1))
reps = np.tile(np.repeat(rep_labels, num_tp), num_cond * num_contr)

# Since condition and contrast are actually separate variables, we'll
#  break them out here.
contrs = np.tile(np.repeat(contr_labels, num_tp * num_reps), num_cond)
conditions = np.repeat(cond_labels, num_tp * num_reps * num_contr)

neuron_labels = ['m1_6', 'm1_12', 'm3_4', 'm3_11', 'm6_3a2', 'm6_11']
num_neurons = len(neuron_labels)

In [19]:
num_condcontr

20

Below is the code that creates the DataFrame. 

Building a DataFrame with 2.5 m rows takes some time, so be patient while this runs:

In [5]:
df_list = []

for n in range(num_neurons):
    neurons = np.repeat(neuron_labels[n], len_data)
    df_tmp = pd.DataFrame(zip(neurons, times, reps, contrs, conditions,
                              dat['SaveForAaron_May11_2020'][0][n][1].reshape(-1, order='F')[None][0]), 
                      columns = ['neuron', 'time', 'repetition', 'contrast', 'condition', 'spike']
                     )
    df_list.append(df_tmp)
    
df = pd.concat(df_list)

# Clear things from memory so CoCalc doesn't run out
del df_tmp
del dat

<font color='#0F4C81'>
<h2> 
    Q5
    </h2>
    
Show the DataFrame, `df`:
</font>

In [0]:
df

### Confirm this worked

The code above is pretty complex, and it is easy to mess it up. It took me many tries to get it right. We can (and should) validate that this worked by checking at the rows in the DataFrame where we expect labels to switch (e.g., at the end of the first rep, end of all reps of first contrast, etc.). You can figure out which rows of the DataFrame to look at using simple multiplication.

<font color='#0F4C81'>
<h2> 
    Q6
    </h2>
Show the rows of the DataFrame corresponding to the end of the first repetition of the first contrast level and the start of the second repetition of the first contrast level (for the first neuron, and the first condition). Show at least two rows before and after the transition from repetition 1 to 2. Hint: Each repetition is 4000 time points, i.e., 4000 rows.
</font>

In [14]:
df[num_tp-2:num_tp+2]

,neuron,time,repetition,contrast,condition,spike
3998,m1_6,3998,1,4,CTRL,0
3999,m1_6,3999,1,4,CTRL,0
4000,m1_6,0,2,4,CTRL,0
4001,m1_6,1,2,4,CTRL,0


<font color='#0F4C81'>
<h2> 
    Q7
    </h2>
Show the rows of the DataFrame corresponding to the end of the 8th repetition of the first contrast level, and the 1st repetition of the second contrast level (for the first neuron, and the first condition). Show at least two rows before and after the transition.
</font>

In [16]:
df[(num_reps*num_tp-2):(num_reps*num_tp+2)]

,neuron,time,repetition,contrast,condition,spike
31998,m1_6,3998,8,4,CTRL,0
31999,m1_6,3999,8,4,CTRL,0
32000,m1_6,0,1,8,CTRL,0
32001,m1_6,1,1,8,CTRL,0


<font color='#0F4C81'>
<h2> 
    Q8
    </h2>
Show the rows of the DataFrame corresponding to the end of the CTRL condition and the start of the ADAPT condition, for the first neuron. Show at least two rows before and after the transition.
</font>

In [18]:
# Find the index of the following
# df[(df['neuron']=='m1_6') & (df['condition'] =='CTRL')].tail(2)
df[(num_contr*num_reps*num_tp-2):(num_contr*num_reps*num_tp+2)]

,neuron,time,repetition,contrast,condition,spike
319998,m1_6,3998,8,100,CTRL,0
319999,m1_6,3999,8,100,CTRL,0
320000,m1_6,0,1,4,ADAPT,0
320001,m1_6,1,1,4,ADAPT,0


<font color='#0F4C81'>
<h2> 
    Q9
    </h2>
Show the rows of the DataFrame corresponding to the end of the data for the first neuron, and the start of the data for the second neuron. Show at least two rows before and after the transition.
</font>

In [20]:
df[(num_condcontr*num_reps*num_tp-2):(num_condcontr*num_reps*num_tp+2)]

,neuron,time,repetition,contrast,condition,spike
639998,m1_6,3998,8,100,ADAPT,0
639999,m1_6,3999,8,100,ADAPT,0
0,m1_12,0,1,4,CTRL,0
1,m1_12,1,1,4,CTRL,0


<font color='#0F4C81'>
<h2> 
    Q10
    </h2>
Show the total number of spikes in the DataFrame:
</font>

In [0]:
df['spike'].sum()

<font color='#0F4C81'>
<h2> 
    Q11
    </h2>

## Raster plots

This is a complex experiment so we need lots of raster plots to see all trials for all contrast levels, and both conditions. So many, in fact (8 x 10 * 2) that we will only want to visualize the rasters for one neuron at a time. 

Below is a copy of the code near the top of this notebook that plotted the experimental design. Modify it to instead plot the rasters of spike times for the first neuron, using the same layout. Remove the horizontal lines that indicated each repetition.
    </font>

In [0]:
fig = plt.figure(figsize=[8,18])

# Assign value first neuron
neuron = df.iloc[0]['neuron']


fig.suptitle('Raster plots of spiking in neuron ' + neuron + ' in each condition and contrast level', fontsize=16)

# This cell runs way faster if we pre-select the neuron rather than using a 
#  combination of '&'' statements inside the loop. Just pre-selecting the neuron
#  dropped the execution time from nearly 5 min to 9 sec!!
neu_dat = df[(df['neuron'] == neuron)]

subplot_counter = 1 # used to track subplots
for contr in contr_labels:
    tmp_dat = neu_dat[(neu_dat['contrast'] == contr)] # this further speeds execution by a factor of 3

    for cond in cond_labels:
        ax = fig.add_subplot(len(contr_labels), len(cond_labels), subplot_counter)
        for rep in rep_labels:
            # insert code that will select the times corresponding to the rows of 
            #  tmp_dat that match the current condition and rep, and also contain spikes
            spike_times = tmp_dat[(tmp_dat['repetition'] == rep) & (tmp_dat['condition'] == cond) & (tmp_dat['spike'] == 1)]['time']
            # change the line below to plot vertical lines at the times of each spike
            plt.vlines(spike_times, rep-1, rep, color='black', linestyle='--', linewidth=.75)

        # Show adaptation grating at 50% contrast
        if cond == 'ADAPT':
            plt.axvspan(0, stim_on_time-1, alpha=0.5, color='grey')
                                                          
        # shading indicates stimulus on period
        plt.axvspan(stim_on_time, stim_off_time, alpha=contr/100, color='grey')
        
        # Pretty formatting - pay attention here, as you may need to change some things
        # to correspond to the raster plot details
        plt.xlim([0, max(time_labels)+1])
        plt.ylim([0, len(rep_labels)])
        if contr == 4:
            plt.title(cond + ' condition')
        
        if contr == 100:
            plt.xlabel('Time (ms)')
        else:
            plt.xticks([])
        
        if cond == 'CTRL':
            plt.ylabel(str(contr) + '% contrast \nRepetition #')
            
        # Should this be modified?
        plt.yticks([x + 0.5 for x in range(num_reps)], [str(x + 1) for x in range(num_reps)], size=8) 
        plt.tight_layout(rect=[0, 0, 1, 0.97])
        subplot_counter += 1
        
plt.show()

<font color='#0F4C81'>
<h2> 
    Q12
    </h2>

## Interactive plot
    
Adapt your code from the cell above to generate an interactive plot, that allows you to select the neuron from a drop-down menu and plot the rasters for that neuron.
    </font>

In [0]:
neurons = df['neuron'].unique()

@interact
def rasterplot_i(neuron=neurons):
    fig = plt.figure(figsize=[8,18])
    fig.suptitle('Raster plots of spiking in neuron ' + neuron + ' in each condition and contrast level')


# This cell runs way faster if we pre-select the neuron rather than using a 
#  combination of '&'' statements inside the loop. Just pre-selecting the neuron
#  dropped the execution time from nearly 5 min to 9 sec!!
    neu_dat = df[(df['neuron'] == neuron)]

    subplot_counter = 1 # used to track subplots
    for contr in contr_labels:
        tmp_dat = neu_dat[(neu_dat['contrast'] == contr)] # this further speeds execution by a factor of 3

        for cond in cond_labels:
            ax = fig.add_subplot(len(contr_labels), len(cond_labels), subplot_counter)
            for rep in rep_labels:
            # insert code that will select the times corresponding to the rows of 
            #  tmp_dat that match the current condition and rep, and also contain spikes
                spike_times = tmp_dat[(tmp_dat['repetition'] == rep) & (tmp_dat['condition'] == cond) & (tmp_dat['spike'] == 1)]['time']
            # change the line below to plot vertical lines at the times of each spike
                plt.vlines(spike_times, rep-1, rep, color='black', linestyle='--', linewidth=.75)

        # Show adaptation grating at 50% contrast
            if cond == 'ADAPT':
                plt.axvspan(0, stim_on_time-1, alpha=0.5, color='grey')
                                                          
        # shading indicates stimulus on period
            plt.axvspan(stim_on_time, stim_off_time, alpha=contr/100, color='grey')
        
        # Pretty formatting - pay attention here, as you may need to change some things
        # to correspond to the raster plot details
            plt.xlim([0, max(time_labels)+1])
            plt.ylim([0, len(rep_labels)])
            if contr == 4:
                plt.title(cond + ' condition')
            if contr == 100:
                plt.xlabel('Time (ms)')
            else:
                plt.xticks([])
            if cond == 'CTRL':
                plt.ylabel(str(contr) + '% contrast \nRepetition #')
        # Should this be modified?
            plt.yticks([x + 0.5 for x in range(num_reps)], [str(x + 1) for x in range(num_reps)], size=8) 
            plt.tight_layout(rect=[0, 0, 1, 0.97])
            subplot_counter += 1
        
    plt.show()
    

<font color='#0F4C81'>
<h2> 
    Q13
    </h2>
  
## PSTHs

Adapt your code from the cell above to generate an interactive peri-stimulus time histogram (PTSH) plot, that allows you to select the neuron from a drop-down menu and plot the PSTH for that neuron. Use the same subplot layout of 2 columns (conditions) and 10 rows (contrast)
    </font>

In [21]:
# DO NOT MODIFY THIS CELL
# Define bins for histograms
hist_bin_width = 50 
time_bins = np.arange(0, max(time_labels), hist_bin_width)

In [0]:
# PUT YOUR ANSWER CODE IN THIS CELL
@interact
def psth_i(neuron=neuron_labels):
    fig = plt.figure(figsize=[8,18])
    fig.suptitle('Peristimulus time histograms of spiking in neuron ' + neuron + ' in each condition and contrast level')

    neu_dat = df[(df['neuron'] == neuron)]

    subplot_counter = 1
    for contr in contr_labels:
        tmp_dat = neu_dat[(neu_dat['contrast'] == contr)]

        for cond in cond_labels:
            ax = fig.add_subplot(len(contr_labels), len(cond_labels), subplot_counter)
            spike_times = []
            for rep in rep_labels:
                spike_times = tmp_dat[(tmp_dat['repetition'] == rep) & (tmp_dat['condition'] == cond) & (tmp_dat['spike'] == 1)]['time']
            np.histogram(spike_times)
            #!!! plt.plot(spike_times, rep-1, rep, color='black', linestyle='--', linewidth=.75)
            # plot time vs. probability of count
            # 0-4000 vs. AP/8

        # Display adaptation grating at 50% contrast
            if cond == 'ADAPT':
                plt.axvspan(0, stim_on_time-1, alpha=0.5, color='grey')
                                                          
        # shading indicates stimulus on period
            plt.axvspan(stim_on_time, stim_off_time, alpha=contr/100, color='grey')
        
        # Pretty formatting - pay attention here, as you may need to change some things
        # to correspond to the raster plot details
            plt.xlim([0, max(time_labels)+1])
            plt.ylim([0, len(rep_labels)])
            if contr == 4:
                plt.title(cond + ' condition')
            if contr == 100:
                plt.xlabel('Time (ms)')
            else:
                plt.xticks([])
            if cond == 'CTRL':
                plt.ylabel(str(contr) + '% contrast \nRepetition #')
        # Should this be modified?
            plt.yticks([x + 0.5 for x in range(num_reps)], [str(x + 1) for x in range(num_reps)], size=8) 
            plt.tight_layout(rect=[0, 0, 1, 0.97])
            subplot_counter += 1
        
    plt.show()

<font color='#0F4C81'>
<h2> 
    Q14
    </h2>

### Heat maps
    
Plot heat maps showing the PTSHs, with colour indicating the spike count. The great thing about heat maps is how condensed they are. We don't really need to use an interactive plot, because one plot includes all contrast levels. So plot this with two columns (CTRL, ADAPT) and 4 rows (each neuron).
    
For each heat map, put time on the *x* axis, contrast on the *y* axis, and colour indicating the number of spikes per bin. 

For heat maps you need to compute the histograms for all contrast levels, and store these temporarily in an object such as a list dictionary, or numpy array.
</font>


In [0]:
cmap='viridis'

<font color='#0F4C81'>
<h2> 
    Q15
    </h2>
    
Provide a rationale for your choice of color map for the heat maps above.
</font>


<font color='green'>
- perceptually uniform
- x% of population with colourblindness

<font color='#0F4C81'>
<h2> 
    Q16
    </h2>
    
## Contrast Response Functions

Plot stimulus contrast (*x*) against the mean spike rate (*y*) during the stimulus "on" period (2000–3000 ms) to produce a contrast response function (CRF), which is a roughly sigmoid-shaped function.

Plot the CRF for the two conditions on the same plot, with one subplot per neuron. 
    </font>

<font color='#0F4C81'>
<h2> 
    Q17
    </h2>

## Neural Latency (latency to first spike)
You can measure neural latency in the control condition, and latency is expected to decrease with increasing contrast.

Using a format similar to the CRFs, make a plot of latency to first spike (computed as the mean across repetitions) on the *y* axis, against contrast on the *x* axis, using one subplot per neuron (with both conditions in one subplot).
</font>

# THE END